In [23]:
print("ok")

ok


In [24]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [26]:
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [27]:
extracted_data = load_pdf_file("/Users/aadesh/Downloads/Doc/Doc/data")

In [28]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [29]:
text_chunks = text_split(extracted_data)
print(len(text_chunks))

7023


In [30]:
from langchain.embeddings import HuggingFaceEmbeddings

In [31]:
def downloadHuggingFaceModel():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [37]:
embeddings = downloadHuggingFaceModel()

In [38]:
query_result = embeddings.embed_query("Hello World")
print("length", len(query_result))

length 384


In [52]:
import os

load_dotenv()

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from dotenv import load_dotenv


pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

pc.create_index(index_name, dimension=384, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1"))

In [54]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [42]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
) 



In [43]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [44]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [47]:
retireved_docs = retriever.invoke("What is Acne?")

In [48]:
retireved_docs

[Document(id='944edcd4-4984-49a7-9a3f-6fe6df9122ae', metadata={'page': 425.0, 'page_label': '426', 'source': '/Users/aadesh/Downloads/Doc/Doc/data/MedBook.pdf'}, page_content='Corticosteriod —A group of synthetic hormones\nthat are used to prevent or reduce inflammation.\nToxic effects may result from rapid withdrawal after\nprolonged use or from continued use of large doses.\nPatch test—A skin test that is done to identify aller-\ngens. A suspected substance is applied to the skin.\nAfter 24–48 hours, if the area is red and swollen,\nthe test is positive for that substance. If no reaction\noccurs, another substance is applied. This is con-'),
 Document(id='99ca4e08-9a4f-42ad-aa99-ad80c4661844', metadata={'page': 298.0, 'page_label': '299', 'source': '/Users/aadesh/Downloads/Doc/Doc/data/MedBook.pdf'}, page_content='Corticosteroids —A group of anti-inflammatory\nsubstances often used to treat skin conditions.\nImmune response—The protective reaction by the\nimmune system against foreig

In [55]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4,max_tokens = 500)

In [59]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for medical question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you aren't sure and should contact a professional. "
    " Use three sentences maximum and keep the answer concise, answer in a "
    "professional doctor-like manner."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [60]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [61]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}